In [ ]:
# -*- coding: utf-8 -*-

3 v3.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Rs-uUxX7UxHJcf6SgqUfbEyGc6HFCnLE

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCVfrom sklearn.pipeline import Pipelinefrom sklearn.preprocessing import StandardScalerfrom sklearn.linear_model import LinearRegressionfrom sklearn.neighbors import KNeighborsRegressorfrom sklearn.tree import DecisionTreeRegressorfrom sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressorfrom sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Para escalar el targetfrom sklearn.preprocessing import StandardScaler as TargetScaler

In [ ]:
pd.set_option("display.max_columns", None)pd.set_option("display.width", 120)

In [ ]:
CSV_PATH = "BTCUSDT_1d_last_year.csv"

In [ ]:
df = pd.read_csv(CSV_PATH)df.columns = [c.lower() for c in df.columns]

In [ ]:
ohlc = ["open","high","low","close"]missing = [c for c in ohlc if c not in df.columns]if missing:    raise ValueError(f"Faltan columnas OHLC: {missing}")

In [ ]:
# Cambios porcentuales (%)for c in ohlc:    df[f"{c}_pct"] = df[c].pct_change()*100.0

In [ ]:
# Volumen opcionalvol_cols = [c for c in ["volume","volumen"] if c in df.columns]vcol = vol_cols[0] if vol_cols else Noneif vcol:    df[f"{vcol}_pct"] = df[vcol].pct_change()*100.0

In [ ]:
# Target: retorno % del próximo período (cierre)df["close_pct_next"] = df["close_pct"].shift(-1)

In [ ]:
df.head()

## 2) *Feature engineering* (lags, medias móviles, volatilidad)
- Lags de `close_pct` (1..5)
- `ma3`, `ma7`: medias móviles de `close_pct`
- `volatilidad_7`: desvío estándar móvil (7)

In [ ]:
# LagsN_LAGS = 5for k in range(1, N_LAGS+1):    df[f"close_pct_lag{k}"] = df["close_pct"].shift(k)

In [ ]:
# Medias móviles y volatilidaddf["ma3"] = df["close_pct"].rolling(3).mean()df["ma7"] = df["close_pct"].rolling(7).mean()df["ma14"] = df["close_pct"].rolling(14).mean()  # Media móvil más largadf["volatilidad_7"] = df["close_pct"].rolling(7).std()df["volatilidad_14"] = df["close_pct"].rolling(14).std()  # Volatilidad más larga

In [ ]:
# Momentum y tendenciadf["momentum_3"] = df["close_pct"] - df["close_pct"].shift(3)  # Diferencia de 3 períodosdf["momentum_7"] = df["close_pct"] - df["close_pct"].shift(7)  # Diferencia de 7 períodos

In [ ]:
# RSI simplificado (indicador de sobrecompra/sobreventa)delta = df["close_pct"]gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()rs = gain / (loss + 1e-10)  # Evitar división por cerodf["rsi_14"] = 100 - (100 / (1 + rs))

In [ ]:
# Feature set base (agregamos las nuevas features)feature_cols = ["open_pct","high_pct","low_pct","close_pct",                "ma3","ma7","ma14",                "volatilidad_7","volatilidad_14",                "momentum_3","momentum_7","rsi_14"]if vcol:    feature_cols.append(f"{vcol}_pct")

In [ ]:
# + lagsfeature_cols += [f"close_pct_lag{k}" for k in range(1, N_LAGS+1)]

In [ ]:
# Drop NaN de pct_change/shift/rollingdf_model = df.dropna(subset=feature_cols + ["close_pct_next"]).copy()

In [ ]:
X = df_model[feature_cols]y = df_model["close_pct_next"]

In [ ]:
X.shape, y.shape, X.head()

## 3) Split temporal y **escalado del *target***
Usamos el 20% final como *test* y escalamos **solo con *train***.  
Luego **desescalamos** las predicciones para evaluar en % real.

In [ ]:
n = len(df_model)cut = int(n*0.8)

In [ ]:
X_train, X_test = X.iloc[:cut], X.iloc[cut:]y_train, y_test = y.iloc[:cut], y.iloc[cut:]

In [ ]:
# Escalado del targetscaler_y = TargetScaler()y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1,1)).ravel()y_test_scaled = scaler_y.transform(y_test.values.reshape(-1,1)).ravel()

In [ ]:
len(X_train), len(X_test)

## 4) Modelos y *grid search* con **TimeSeriesSplit**
Incluimos ahora un **RandomForestRegressor**.

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
models = {    "DecisionTreeRegressor": {    "pipe": Pipeline([("model", DecisionTreeRegressor(random_state=42))]),    "param_grid": {        "model__max_depth": [15, 20, 25, None],  # Árboles más profundos        "model__min_samples_leaf": [1, 2, 3],    # Permitir hojas más pequeñas        "model__min_samples_split": [2, 3, 5],   # Más flexible para dividir        "model__min_impurity_decrease": [0.0, 0.001]  # Control de crecimiento        }    },    "RandomForestRegressor": {        "pipe": Pipeline([("model", RandomForestRegressor(random_state=42, n_jobs=-1))]),        "param_grid": {            "model__n_estimators": [100, 200, 300],  # Más árboles            "model__max_depth": [10, 15, 20, None],  # Árboles más profundos            "model__min_samples_leaf": [1, 2, 3],            "model__max_features": ["sqrt", "log2", 0.5]  # Variedad en features por split        }    },    "GradientBoostingRegressor": {        "pipe": Pipeline([("model", GradientBoostingRegressor(random_state=42))]),        "param_grid": {            "model__n_estimators": [100, 200, 300],            "model__max_depth": [3, 5, 7],            "model__learning_rate": [0.01, 0.05, 0.1],            "model__min_samples_leaf": [1, 2, 4]        }    }}

In [ ]:
results = []best_estimators = {}

In [ ]:
for name, cfg in models.items():    pipe = cfg["pipe"]    grid = cfg["param_grid"]    if grid:        search = GridSearchCV(            estimator=pipe,            param_grid=grid,            scoring="neg_mean_absolute_error",            cv=tscv,            n_jobs=-1        )        search.fit(X_train, y_train_scaled)  # usamos target escalado        best = search.best_estimator_        best_estimators[name] = best        cv_mae = -search.best_score_        cv_params = search.best_params_    else:        pipe.fit(X_train, y_train_scaled)        best = pipe        best_estimators[name] = best        cv_mae = None        cv_params = {}

In [ ]:
# Predicción en test (escala del target)    preds_scaled = best.predict(X_test)    # Desescalar a %    preds = scaler_y.inverse_transform(np.array(preds_scaled).reshape(-1,1)).ravel()

In [ ]:
mae = mean_absolute_error(y_test, preds)    rmse = np.sqrt(mean_squared_error(y_test, preds))    r2 = r2_score(y_test, preds)

In [ ]:
results.append({        "modelo": name,        "cv_mae": cv_mae,        "mejores_params": cv_params,        "test_mae": mae,        "test_rmse": rmse,        "test_r2": r2    })

In [ ]:
res_df = pd.DataFrame(results).sort_values("test_mae")res_df

## 5) Curva Real vs Predicción del mejor modelo (COMENTADO PARA NO DEMORAR)

In [ ]:
# best_name = res_df.iloc[0]["modelo"]# best_model = best_estimators[best_name]

In [ ]:
# preds_scaled = best_model.predict(X_test)# preds = scaler_y.inverse_transform(np.array(preds_scaled).reshape(-1,1)).ravel()

In [ ]:
# plt.figure(figsize=(10,4))# plt.plot(y_test.values, label="Real")# plt.plot(preds, label=f"Predicción ({best_name})")# plt.title("Cambio % de close (siguiente período)")# plt.legend()# plt.xlabel("Observación (orden temporal)")# plt.ylabel("%")# plt.tight_layout()# plt.show()

In [ ]:
res_df

In [ ]:
# VISUALIZACIÓN MÚLTIPLE COMENTADA (descomentar si querés ver todos los modelos)# model_names = ["DecisionTreeRegressor", "RandomForestRegressor", "GradientBoostingRegressor"]# fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True, sharey=True)

In [ ]:
# for i, name in enumerate(model_names):#     if name not in best_estimators:#         continue

In [ ]:
#     model = best_estimators[name]#     preds_scaled = model.predict(X_test)#     preds = scaler_y.inverse_transform(np.array(preds_scaled).reshape(-1,1)).ravel()

In [ ]:
#     ax = axes[i]#     ax.plot(y_test.values, label="Real", linewidth=1.8)#     ax.plot(preds, label=f"Predicción ({name})", linewidth=1.5)#     ax.set_title(f"{name} – Cambio % de close (siguiente período)")#     ax.set_ylabel("%")#     ax.legend(loc="upper right")

In [ ]:
# axes[-1].set_xlabel("Observación (orden temporal)")# plt.tight_layout()# plt.show()

## 6) Exportar el mejor modelo con metadata completa

In [ ]:
import joblib

In [ ]:
# Obtener el mejor modelobest_name = res_df.iloc[0]["modelo"]best_model = best_estimators[best_name]

In [ ]:
# Extraer solo el modelo (sin el pipeline de scaling si existe)if hasattr(best_model, 'named_steps'):    final_model = best_model.named_steps['model']else:    final_model = best_model

In [ ]:
# Crear artefacto completo con TODA la información necesariaartifact = {    "model": final_model,    "model_name": best_name,    "feature_names": feature_cols,  # Lista ordenada de features    "base_features": ["close", "volume", "high", "low", "open"],    "n_lags": N_LAGS,    "use_feedback": False,  # Si usas feedback, cambiar a True    "scaler_y": scaler_y,   # Scaler del target    "metrics": {        "test_mae": res_df.iloc[0]["test_mae"],        "test_rmse": res_df.iloc[0]["test_rmse"],        "test_r2": res_df.iloc[0]["test_r2"]    },    "best_params": res_df.iloc[0]["mejores_params"]}

In [ ]:
# Guardarjoblib.dump(artifact, "models/model_feedback.pkl")print(f"✅ Modelo {best_name} guardado en models/model_feedback.pkl")print(f"   Features: {len(feature_cols)}")print(f"   Test MAE: {artifact['metrics']['test_mae']:.4f}%")print(f"   Test R²: {artifact['metrics']['test_r2']:.4f}")

In [ ]:
# Info del árbol si es DecisionTree o RandomForestif hasattr(final_model, 'tree_'):    print(f"   Árbol - Hojas: {final_model.tree_.n_leaves}, Profundidad: {final_model.tree_.max_depth}")elif hasattr(final_model, 'estimators_'):    if len(final_model.estimators_) > 0 and hasattr(final_model.estimators_[0], 'tree_'):        avg_leaves = np.mean([tree.tree_.n_leaves for tree in final_model.estimators_])        print(f"   Random Forest - Árboles: {len(final_model.estimators_)}, Promedio hojas: {avg_leaves:.0f}")